In [1]:
#Adapted from "2025-03-28 Using sextupole movers to control two-bunch pointing.ipynb"
from UTILITY_quickstart import *

In [2]:
importedDefaultSettings = loadConfig("setLattice_configs/2024-10-14_twoBunch_baseline.yml")

In [3]:
csrTF = True
evalElement = "PENT"
#evalElement = "BEGPDC"


inputBeamFilePathSuffix = importedDefaultSettings["inputBeamFilePathSuffix"]
bunchCount = importedDefaultSettings["bunchCount"]
tao = initializeTao(
    inputBeamFilePathSuffix = inputBeamFilePathSuffix,
    
    csrTF = csrTF,
    numMacroParticles=1e5,
    scratchPath = "/tmp",
    randomizeFileNames = True
)

setLattice(tao, **importedDefaultSettings)

Environment set to:  /Users/nmajik/Documents/SLAC/FACET2-Bmad-PyTao
Tracking to end
CSR on
Overwriting lattice with setLattice() defaults
No defaults file provided to setLattice(). Using setLattice_configs/defaults.yml
Number of macro particles = 100000.0
Beam created, written to /tmp/beams/activeBeamFile_9929528010727385033.h5, and reinit to tao


In [4]:
def getBeamVectors(P):
    PDrive, PWitness = getDriverAndWitness(P) 
    return [ 
        np.median(PDrive.x),
        np.median(PDrive.xp),
        np.median(PDrive.y),
        np.median(PDrive.yp),
        np.median(PWitness.x),
        np.median(PWitness.xp),
        np.median(PWitness.y),
        np.median(PWitness.yp)
    ]
    

## Consider the original, unmodified config

In [5]:
trackBeam(tao, 
          **importedDefaultSettings,
          verbose = True
         )

P = getBeamAtElement(tao, "PENT")
getBeamVectors(P)

Loaded /tmp/beams/activeBeamFile_9929528010727385033.h5
Set track_start = L0AFEND, track_end = end
Tracking!
trackBeam() exiting


[1.275743701524768e-05,
 -3.953026856488402e-05,
 1.988418244099759e-05,
 -0.0001241212499885796,
 -1.1338030055798323e-05,
 2.341817662812979e-05,
 1.6027301398963385e-05,
 -0.0002718422940297606]

In [6]:
PStart = getBeamAtElement( tao, "BEGBC20" ) 

makeBeamActiveBeamFile(PStart, tao = tao)

trackBeam(tao, trackStart = "BEGBC20", **importedDefaultSettings)

P = getBeamAtElement(tao, "PENT")
getBeamVectors(P)

[1.2735471207492383e-05,
 -3.948392133558717e-05,
 1.9879877441638787e-05,
 -0.00012411506461010514,
 -1.1353897542040972e-05,
 2.3382620308433594e-05,
 1.6031533703265795e-05,
 -0.0002718581399402147]

## Baseline, sextupoles centered

In [7]:
centerSextupolesDict = {
    "S1EL_xOffset" : 0.0,
    "S1EL_yOffset" : 0.0,
    "S2EL_xOffset" : 0.0,
    "S2EL_yOffset" : 0.0,
    "S2ER_xOffset" : 0.0,
    "S2ER_yOffset" : 0.0,
    "S1ER_xOffset" : 0.0,
    "S1ER_yOffset" : 0.0,   
}

setLattice(tao, **( importedDefaultSettings | centerSextupolesDict) )

trackBeam(tao, trackStart = "BEGBC20", **importedDefaultSettings)

P = getBeamAtElement(tao, "PENT")

baselineVectors = getBeamVectors(P)
baselineVectors

resultsForExport = [ baselineVectors ] 

## Perturb each quad

In [8]:
configQuadSettings = {
    key : importedDefaultSettings[key] for key in ["Q1EkG", "Q2EkG", "Q3EkG", "Q4EkG", "Q5EkG", "Q6EkG"]
}

configQuadSettings

{'Q1EkG': 161.311,
 'Q2EkG': -154.229,
 'Q3EkG': 110.217,
 'Q4EkG': 132.268,
 'Q5EkG': -23.373,
 'Q6EkG': -142.271}

In [9]:
for key in configQuadSettings.keys():
    #setLattice(tao, **( importedDefaultSettings | centerSextupolesDict | {key : 100e-6} ) )
    setLattice(tao, **( importedDefaultSettings | centerSextupolesDict | {key : (configQuadSettings[key] + 1 )} ) )

    trackBeam(tao, trackStart = "BEGBC20", **importedDefaultSettings)
    
    P = getBeamAtElement(tao, "PENT")

    newVectors = getBeamVectors(P)

    resultsForExport.append(newVectors)
    print( newVectors )

[-9.22119131001791e-05, -6.572692472962236e-05, -4.984242570072535e-05, -7.372288525183376e-05, -0.00031817055668453993, -0.00028084187200037, -6.96570082592413e-05, -8.12390919345578e-05]
[-9.927435873815173e-05, -5.871223338439244e-05, -4.989492039014285e-05, -7.48178859368769e-05, -0.0003425245280101178, -0.0002462979735715825, -6.931422691107452e-05, -8.115703842218452e-05]
[-0.00023285542901255383, -1.6927290290600252e-05, -4.887003365958842e-05, -7.4397734359747e-05, -0.000843580734213964, 0.00017401733292900524, -6.975594530647536e-05, -8.13528513995524e-05]
[-0.00010919132122946167, -5.950258241564577e-05, -4.967355870172373e-05, -7.39228910948573e-05, -0.0003645245833149542, -0.00023816476554058372, -6.963895458935507e-05, -8.113775593640103e-05]
[-9.077016112688722e-05, -5.93360626798936e-05, -4.9866246603213225e-05, -7.426042279312118e-05, -0.0003086265759801061, -0.00026405100524108064, -6.942194392489965e-05, -8.084756075012259e-05]
[-8.789833847032751e-05, -5.642001298105

In [10]:
resultsForExport = np.array(resultsForExport)
np.savetxt("/Users/nmajik/vectors.csv", resultsForExport, delimiter=',')